In [ ]:
# import torch
# import torchaudio

# temp, _ = torchaudio.load('/content/drive/MyDrive/MIDI4STRINGS_but_better/temp.wav')

# temp = torch.Tensor(temp)

# print(temp.shape)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Define your LSTM model
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, output_size):
#         super(LSTMModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

#         out, _ = self.lstm(x, (h0, c0))

#         out = self.fc(out[:, -1, :])
#         out = self.sigmoid(out)
#         return out

# # Example usage
# input_size = (100,2) # define your input size based on your feature extraction
# hidden_size = 128
# num_layers = 2
# output_size = 1

# model = LSTMModel(input_size, hidden_size, num_layers, output_size)


# # # Define loss and optimizer
# # criterion = nn.BCELoss()
# # optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Example data loading and training loop

# def prepare_dataset(human_files, synthesized_files):
#     features = []
#     labels = []

#     for file in human_files:
#         features.append(extract_features(file))
#         labels.append(0.0)  # 0.0 for human

#     for file in synthesized_files:
#         features.append(extract_features(file))
#         labels.append(1.0)  # 1.0 for synthesized

#     features = np.array(features)
#     labels = np.array(labels)
#     return train_test_split(features, labels, test_size=0.0001, random_state=14)

# # Example file lists
# human_files = []
# synthesized_files = []
# for i in range(10):
#   human_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human_{i}.mp3")
#   synthesized_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/MIDI_{i}.mp3")

# # Emphasize lack of importance of silent time at the beginning
# for i in range(3):
#   synthesized_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/MIDI_{4}.mp3")
#   human_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human_{5}.mp3")

# X_train, X_test, y_train, y_test = prepare_dataset(human_files, synthesized_files)

# # Training the model
# def train_model(model, X_train, y_train, X_test, y_test, epochs=1000, learning_rate=0.0001):
#     criterion = nn.BCELoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#     train_losses = []
#     test_losses = []

#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         outputs = model(X_train)
#         # if (epoch == 1):
#         #   print(outputs.shape, y_train.shape)
#         loss = criterion(outputs, y_train)
#         loss.backward()
#         optimizer.step()
#         train_losses.append(loss.item()) # Records the losses for visualization

#         if (epoch+1) % 100 == 0:
#             model.eval()
#             test_outputs = model(X_test)
#             test_loss = criterion(test_outputs, y_test)
#             test_losses.append(test_loss.item())
#             print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Test Loss: {test_loss.item()}')

#     return train_losses, test_losses

# train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test, epochs=2000, learning_rate=0.0001) # Remove first half of equation if no visualization

# # Visualize
# def plot_losses(train_losses, test_losses):
#     plt.figure(figsize=(10, 5))
#     plt.plot(train_losses, label='Train Loss')
#     plt.plot(test_losses, label='Test Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.title('Training and Test Loss Over Epochs')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# plot_losses(train_losses, test_losses)

# # Save the model
# def save_model(model, path):
#     torch.save(model.state_dict(), path)
#     print(f"Model saved to {path}")

# save_path = "audio_regressor_model.pth"
# save_model(model, save_path)
# --------------------------------------------------------------------------------------------------------------------------------------------------
# # Assuming you have a DataLoader `train_loader` containing your data

# # num_epochs = 10
# # for epoch in range(num_epochs):
# #     for i in range(len(X_train)):
# #         optimizer.zero_grad()
# #         outputs = model(X_train[i])
# #         loss = criterion(outputs, y_train[i])
# #         loss.backward()
# #         optimizer.step()

# #     # Validation after each epoch if needed

# # # Evaluate your model on test data
# # # Assuming you have a separate DataLoader `test_loader` for test data
# # model.eval()
# # with torch.no_grad():
# #     for i in range(len(X_test)):
# #         outputs = model(X_test[i])
# #         # Calculate metrics (accuracy, etc.) as needed

In [2]:
import torch
from google.colab import drive
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
 # Feature extraction and data preparation
import librosa
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    features = np.concatenate((mfccs.mean(axis=1), chroma.mean(axis=1), spectral_contrast.mean(axis=1)))
    return features

def prepare_dataset(human_files, synthesized_files):
    features = []
    labels = []

    for file in human_files:
        features.append(extract_features(file))
        labels.append(0.0)  # 0.0 for human

    for file in synthesized_files:
        features.append(extract_features(file))
        labels.append(1.0)  # 1.0 for synthesized

    return (features[:-1], features[-1], labels[:-1], labels[-1]) # Everything is train
    # features = np.array(features)
    # labels = np.array(labels)
    # return train_test_split(features, labels, test_size=0.00001, random_state=14)

# Example file lists
import torch
from os import walk

# Auxilliary Function
# Import MIDI & MP3 Files
synthesized_path = "/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human-ness Training Data/MIDI_Synth/"
synthesized_temp = []
synthesized_files = []
human_path = "/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human-ness Training Data/Human/"
human_temp = []
human_files = []

# Parsing through MIDI files
for (dir_path, dir_names, file_names) in walk(synthesized_path):
  synthesized_temp.extend(file_names)

for file in synthesized_temp:
  midi_file_path = synthesized_path + file
  synthesized_files.append(midi_file_path)

print(synthesized_files) # Check

# Parsing through Human files
for (dir_path, dir_names, file_names) in walk(human_path):
  human_temp.extend(file_names)

for file in human_temp:
  human_file_path = human_path + file
  human_files.append(human_file_path)

print(human_files) # Check

# # Emphasize lack of importance of silent time at the beginning + Human_0 is just so good
# for i in range(2):
#   synthesized_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/MIDI_{4}.mp3")
#   human_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human_{5}.mp3")
#   human_files.append(f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human_{0}.mp3")

X_train, X_test, y_train, y_test = prepare_dataset(human_files, synthesized_files) # For test/train split


# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

[]
[]


IndexError: list index out of range

In [5]:
# Model definition
class AudioRegressor(nn.Module):
    def __init__(self, input_dim):
        super(AudioRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.fc4 = nn.Linear(2, 32)
        self.fc5 = nn.Linear(32, 64)
        self.fc6 = nn.Linear(64,1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.fc6(x)
        return x

input_dim = X_train.shape[1]
model = AudioRegressor(input_dim)

# Training the model
def train_model(model, X_train, y_train, X_test, y_test, epochs=100, learning_rate=0.0001):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        # if (epoch == 1):
        #   print(outputs.shape, y_train.shape)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item()) # Records the losses for visualization

        if (epoch+1) % 1 == 0:
            model.eval()
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test)
            test_losses.append(test_loss.item())
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Test Loss: {test_loss.item()}')

    return train_losses, test_losses

train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test, epochs=10000, learning_rate=0.00001) # Remove first half of equation if no visualization

# Visualize
def plot_losses(train_losses, test_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Test Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_losses(train_losses, test_losses)

# Save the model
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

save_path = "/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human-ness Training Data/human-ness_model.pth"
save_model(model, save_path)


NameError: name 'X_train' is not defined

In [ ]:
import librosa
import numpy as np
import torch
import torch.nn as nn

# Feature extraction
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    features = np.concatenate((mfccs.mean(axis=1), chroma.mean(axis=1), spectral_contrast.mean(axis=1)))
    return features

# Model definition
class AudioRegressor(nn.Module):
    def __init__(self, input_dim):
        super(AudioRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.fc4 = nn.Linear(2, 32)
        self.fc5 = nn.Linear(32, 64)
        self.fc6 = nn.Linear(64,1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.fc6(x)
        return x

# Load the model
def load_model(path, input_dim):
    model = AudioRegressor(input_dim)
    model.load_state_dict(torch.load(path))
    model.eval()  # Set the model to evaluation mode
    print(f"Model loaded from {path}")
    return model

# Predict function
def predict(file_path, model):
    features = extract_features(file_path)
    features = torch.tensor(features, dtype=torch.float32).view(1, -1)
    model.eval()
    with torch.no_grad():
        prediction = model(features)  # Get the output tensor
        prediction = torch.mean(prediction).item()  # Example: Take the mean value
    return prediction

# Example usage
save_path = "/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human-ness Training Data/human-ness_model.pth"
input_dim = 32  # Number of features extracted

# Load the model
loaded_model = load_model(save_path, input_dim)

# Custom prediction
score = predict("/content/drive/MyDrive/MIDI4STRINGS_but_better/temp1.mp3", loaded_model)
print(f"The synthesized score of this custon audio is: {score:.2f}")

# Make prediction
for file in synthesized_files:
  score = predict(file, loaded_model)
  print(f"File name: {file}\nThe synthesized score of this synthesized audio is: {score:.2f}")
for file in human_files:
  score = predict(file, loaded_model)
  print(f"File name: {file}\nThe synthesized score of this human audio is: {score:.2f}")
# for i in range(10):
#   file_path = f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/MIDI_{i}.mp3"
#   score = predict(file_path, loaded_model)
#   print(f"The synthesized score of this synthesized audio is: {score:.2f} MIDI{i}")
# for i in range(10):
#   file_path = f"/content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human_{i}.mp3"
#   score = predict(file_path, loaded_model)
#   print(f"The synthesized score of this human audio is: {score:.2f} Human{i}")

Model loaded from /content/drive/MyDrive/MIDI4STRINGS_but_better/Training Data/Human-ness Training Data/human-ness_model.pth
The synthesized score of this custon audio is: 1.01


NameError: name 'synthesized_files' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')